In [141]:
import warnings
warnings.filterwarnings("ignore")

In [142]:
import numpy as np
import pandas as pd

In [143]:
train = pd.read_csv("C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Data/train.csv")
test = pd.read_csv("C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Data/test.csv")

In [144]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [145]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [146]:
train.loc[6]

id                                                          6
brand                                                    Audi
model                                        S5 3.0T Prestige
model_year                                               2014
milage                                                 107380
fuel_type                                            Gasoline
engine          333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
transmission                                      7-Speed A/T
ext_col                                                  Gray
int_col                                                 Black
accident                                        None reported
clean_title                                               Yes
price                                                   26500
Name: 6, dtype: object

In [147]:
for column in train.select_dtypes(include=['object']).columns:
    print(f"Value counts for column '{column}':")
    print(train[column].value_counts())
    print()

Value counts for column 'brand':
brand
BMW              7369
Ford             6706
Mercedes-Benz    5087
Chevrolet        4424
Audi             2922
Porsche          2627
Toyota           2315
Lexus            2257
Jeep             2247
Land             1995
Cadillac         1565
Nissan           1252
GMC              1076
Dodge            1011
RAM               966
INFINITI          957
Lincoln           767
Mazda             748
Subaru            739
Hyundai           694
Jaguar            661
Volkswagen        628
Honda             624
Acura             580
Kia               526
Volvo             452
MINI              364
Maserati          293
Bentley           284
Chrysler          258
Genesis           249
Buick             228
Mitsubishi        182
Hummer            176
Pontiac           149
Alfa              144
Rolls-Royce       142
Lamborghini       122
Tesla             110
Ferrari            87
Saturn             58
Scion              53
Aston              50
McLaren        

In [148]:
train['int_black'] = train['int_col'].apply(lambda x: 1 if x == 'Black' else 0)
test['int_black'] = test['int_col'].apply(lambda x: 1 if x == 'Black' else 0)

In [149]:
def categorize_color(color):
    color_lower = color.lower()
    if 'black' in color_lower:
        return 'black'
    elif 'white' in color_lower:
        return 'white'
    else:
        return 'other'

train['ext_col_processed'] = train['ext_col'].apply(categorize_color)
test['ext_col_processed'] = test['ext_col'].apply(categorize_color)

In [150]:
# Function to parse the 'engine' column
def parse_engine(engine_str):
    parts = engine_str.split()
    horsepower, displacement, cylinder, engine_type, fuel = (np.nan, np.nan, np.nan, np.nan, np.nan)
    
    try:
        # Parse horsepower
        if 'HP' in parts[0]:
            horsepower = parts[0].replace('HP', '')
            parts.pop(0)
        
        # Parse displacement
        if 'L' in parts[0]:
            displacement = parts[0].replace('L', '')
            parts.pop(0)
        elif 'Liter' in parts[0]:
            displacement = parts[0].replace('Liter', '')
            parts.pop(0)

        # Parse cylinder
        if parts[1] == 'Cylinder':
            cylinder = parts[0]
            parts.pop(0)
            parts.pop(0)
        
        # Parse engine type and fuel
        if len(parts) >= 2:
            engine_type = parts[0]
            fuel = parts[-2] if 'Fuel' in parts[-1] else parts[-1]
    except:
        pass
    
    return pd.Series([horsepower, displacement, cylinder, engine_type, fuel])

# Apply the function to the 'engine' column and create new columns
train[['horsepower', 'displacement', 'cylinder', 'type', 'fuel']] = train['engine'].apply(parse_engine)
test[['horsepower', 'displacement', 'cylinder', 'type', 'fuel']] = test['engine'].apply(parse_engine)

In [151]:
train = train.drop(['id','clean_title','int_col','ext_col','transmission','engine','model','fuel_type'],axis=1)
test = test.drop(['clean_title','int_col','ext_col','transmission','engine','model','fuel_type'],axis=1)

In [152]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   brand              54273 non-null  object
 1   model_year         54273 non-null  int64 
 2   milage             54273 non-null  int64 
 3   accident           54273 non-null  object
 4   price              54273 non-null  int64 
 5   int_black          54273 non-null  int64 
 6   ext_col_processed  54273 non-null  object
 7   horsepower         50216 non-null  object
 8   displacement       53667 non-null  object
 9   cylinder           43555 non-null  object
 10  type               53901 non-null  object
 11  fuel               53901 non-null  object
dtypes: int64(4), object(8)
memory usage: 5.0+ MB


In [153]:
train.head()

,brand,model_year,milage,accident,price,int_black,ext_col_processed,horsepower,displacement,cylinder,type,fuel
0,Ford,2018,74349,None reported,11000,0,other,375.0,3.5,V6,Engine,Gasoline
1,BMW,2007,80000,None reported,8250,1,black,300.0,3.0,NaN,Straight,Gasoline
2,Jaguar,2009,91491,None reported,15000,0,other,300.0,4.2,8,Engine,Gasoline
3,BMW,2022,2437,None reported,63500,0,other,335.0,3.0,NaN,Straight,Hybrid
4,Pontiac,2001,111000,None reported,7850,1,white,200.0,3.8,V6,Engine,Gasoline


In [154]:
# train_cleaned = train.dropna()
# test_cleaned = test.dropna()

In [155]:
# train['model_year'] = train['model_year'].astype('object')
# test['model_year'] = test['model_year'].astype('object')

In [156]:
train_price = train['price']

train = train.drop(['price'],axis=1)


test_id = test['id']

test = test.drop(['id'],axis=1)

In [157]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

def preprocess_data(df):
    # Selecting numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns
    df_numerical = df[numerical_columns]
    
    # Handling missing values for numerical columns using SimpleImputer with mean strategy
    imputer_numerical = SimpleImputer(strategy='mean')
    df_imputed_numerical = pd.DataFrame(imputer_numerical.fit_transform(df_numerical), columns=df_numerical.columns)
    
    # Scaling numerical features using StandardScaler
    scaler = StandardScaler()
    df_scaled_numerical = pd.DataFrame(scaler.fit_transform(df_imputed_numerical), columns=df_imputed_numerical.columns)
    
    # Selecting categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    df_categorical = df[categorical_columns]
    
    # Handling missing values for categorical columns using SimpleImputer with most frequent strategy
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    df_imputed_categorical = pd.DataFrame(imputer_categorical.fit_transform(df_categorical), columns=df_categorical.columns)
    
    # Encoding categorical variables using LabelEncoder
    df_encoded_categorical = df_imputed_categorical.apply(lambda col: LabelEncoder().fit_transform(col))
    
    # Merging numerical and encoded categorical columns
    df_processed = pd.concat([df_scaled_numerical, df_encoded_categorical], axis=1)
    
    return df_processed

# Assuming df is your DataFrame
# Replace 'df' with the name of your DataFrame
train_processed = preprocess_data(train)
test_processed = preprocess_data(test)


In [158]:
train_processed['price'] = train_price
test_processed['id'] = test_id

In [159]:
train.columns

Index(['brand', 'model_year', 'milage', 'accident', 'int_black',
       'ext_col_processed', 'horsepower', 'displacement', 'cylinder', 'type',
       'fuel'],
      dtype='object')

In [160]:
train_processed.head()

,model_year,milage,int_black,brand,accident,ext_col_processed,horsepower,displacement,cylinder,type,fuel,price
0,0.520325,0.031759,-1.190455,14,1,1,203,21,6,15,9,11000
1,-1.447877,0.143728,0.840015,4,1,0,152,17,5,25,9,8250
2,-1.090022,0.371412,-1.190455,21,1,1,152,27,5,15,9,15000
3,1.236035,-1.393115,-1.190455,4,1,1,181,17,5,25,10,63500
4,-2.521442,0.757966,0.840015,40,1,2,73,24,6,15,9,7850


In [161]:
from scipy import stats

# Calculate the Z-scores of each column
z_scores = np.abs(stats.zscore(train_processed))

# Set a threshold for identifying outliers
threshold = 3

# Identify the indices of rows containing outliers
outliers = np.where(z_scores > threshold)
outlier_indices = set(outliers[0])

# Drop the rows with outliers
train_processed = train_processed.drop(outlier_indices).reset_index(drop=True)

In [162]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49892 entries, 0 to 49891
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model_year         49892 non-null  float64
 1   milage             49892 non-null  float64
 2   int_black          49892 non-null  float64
 3   brand              49892 non-null  int32  
 4   accident           49892 non-null  int32  
 5   ext_col_processed  49892 non-null  int32  
 6   horsepower         49892 non-null  int32  
 7   displacement       49892 non-null  int32  
 8   cylinder           49892 non-null  int32  
 9   type               49892 non-null  int32  
 10  fuel               49892 non-null  int32  
 11  price              49892 non-null  int64  
dtypes: float64(3), int32(8), int64(1)
memory usage: 3.0 MB


In [163]:
train_processed.corr()

,model_year,milage,int_black,brand,accident,ext_col_processed,horsepower,displacement,cylinder,type,fuel,price
model_year,1.000000,-0.632555,0.177353,-0.075547,0.217836,0.033020,0.382794,0.008652,0.005952,0.059928,-0.029526,0.503653
milage,-0.632555,1.000000,-0.165155,0.050154,-0.297272,-0.025492,-0.395290,-0.032636,-0.004363,-0.076970,-0.066375,-0.541270
int_black,0.177353,-0.165155,1.000000,-0.042204,0.079752,-0.161983,0.120834,0.006933,-0.020408,-0.009559,0.039939,0.112073
brand,-0.075547,0.050154,-0.042204,1.000000,-0.017235,-0.001926,-0.148872,-0.074738,-0.021581,-0.210662,0.035196,-0.013541
accident,0.217836,-0.297272,0.079752,-0.017235,1.000000,0.004354,0.181727,0.051510,0.044757,0.044265,0.029612,0.231818
ext_col_processed,0.033020,-0.025492,-0.161983,-0.001926,0.004354,1.000000,-0.029766,-0.015471,-0.034943,-0.028830,-0.025600,-0.010533
horsepower,0.382794,-0.395290,0.120834,-0.148872,0.181727,-0.029766,1.000000,0.638226,0.417460,0.038678,-0.008819,0.471654
displacement,0.008652,-0.032636,0.006933,-0.074738,0.051510,-0.015471,0.638226,1.000000,0.423440,-0.133648,-0.191261,0.172687
cylinder,0.005952,-0.004363,-0.020408,-0.021581,0.044757,-0.034943,0.417460,0.423440,1.000000,0.037058,-0.012198,0.091227
type,0.059928,-0.076970,-0.009559,-0.210662,0.044265,-0.028830,0.038678,-0.133648,0.037058,1.000000,-0.116591,0.061264


In [164]:
correlation_matrix = train_processed.corr()

# Threshold for correlation coefficient
threshold = 0.1

# Get the absolute correlation with the target variable 'price'
target_corr = correlation_matrix['price'].abs()

# Select columns with correlation above the threshold
relevant_columns = target_corr[target_corr > threshold].index

# Assuming 'df' is your original DataFrame
# df = df[relevant_columns]

# Display the relevant columns
print("Relevant columns:", relevant_columns)

Relevant columns: Index(['model_year', 'milage', 'int_black', 'accident', 'horsepower',
       'displacement', 'price'],
      dtype='object')


In [165]:
# train_processed = train_processed.drop(['brand','ext_col_processed','cylinder','type','fuel'],axis=1)
# test_processed = test_processed.drop(['brand','ext_col_processed','cylinder','type','fuel'],axis=1)

In [166]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49892 entries, 0 to 49891
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model_year         49892 non-null  float64
 1   milage             49892 non-null  float64
 2   int_black          49892 non-null  float64
 3   brand              49892 non-null  int32  
 4   accident           49892 non-null  int32  
 5   ext_col_processed  49892 non-null  int32  
 6   horsepower         49892 non-null  int32  
 7   displacement       49892 non-null  int32  
 8   cylinder           49892 non-null  int32  
 9   type               49892 non-null  int32  
 10  fuel               49892 non-null  int32  
 11  price              49892 non-null  int64  
dtypes: float64(3), int32(8), int64(1)
memory usage: 3.0 MB


In [167]:
train_processed.describe()

,model_year,milage,int_black,brand,accident,ext_col_processed,horsepower,displacement,cylinder,type,fuel,price
count,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000,49892.000000
mean,-0.006739,0.006573,0.017238,20.819049,0.726269,0.970817,165.267919,22.773030,4.893690,16.240940,8.675539,34557.216247
std,0.939749,0.945068,0.996836,14.802111,0.445877,0.737738,67.847141,11.331573,1.072824,3.318376,1.359965,29650.233815
min,-2.879297,-1.439421,-1.190455,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,5.000000,2.000000,2000.000000
25%,-0.553240,-0.739982,-1.190455,9.000000,0.000000,0.000000,118.000000,17.000000,5.000000,15.000000,9.000000,15499.000000
50%,0.162470,-0.103948,0.840015,17.000000,1.000000,1.000000,159.000000,21.000000,5.000000,15.000000,9.000000,27000.000000
75%,0.699252,0.599453,0.840015,35.000000,1.000000,2.000000,216.000000,29.000000,6.000000,15.000000,9.000000,44000.000000
max,1.593889,2.996963,0.840015,52.000000,1.000000,2.000000,339.000000,57.000000,6.000000,28.000000,16.000000,257000.000000


In [168]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
import xgboost as xgb

# Split the data into features and target variable
X = train_processed.drop(columns='price')  # Features
y = train_processed['price']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
xgb_reg = xgb.XGBRegressor(random_state=42)

# Train the model
xgb_reg.fit(X_train, y_train)

# Make predictions
y_pred = xgb_reg.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 21671.411068571164


In [169]:
import statsmodels.api as sm

X = train_processed.drop(columns='price') 
y = train_processed['price']

X = sm.add_constant(X)

X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=42)

reg = sm.OLS(y_train , X_train).fit()

reg.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Results: Ordinary least squares
============================================================================
Model:                  OLS                Adj. R-squared:       0.406      
Dependent Variable:     price              AIC:                  857799.1796
Date:                   2024-06-06 23:20   BIC:                  857901.5388
No. Observations:       37419              Log-Likelihood:       -4.2889e+05
Df Model:               11                 F-statistic:          2327.      
Df Residuals:           37407              Prob (F-statistic):   0.00       
R-squared:              0.406              Scale:                5.2872e+08 
----------------------------------------------------------------------------
                    Coef.     Std.Err.    t     P>|t|    [0.025     0.975]  
----------------------------------------------------------------------------
const              9152.3239 1278.4443   7.1590 0.0000  6646.5380 11658.1098
model_year         6460.1159  172.2880  37.4960 0.0000  6122.4268  6797.8050
milage            -8920.5261  173.1515 -51.5186 0.0000 -9259.9077 -8581.1444
int_black          -351.4011  123.7255  -2.8402 0.0045  -593.9064  -108.8957
brand               134.1920    8.3319  16.1058 0.0000   117.8612   150.5228
accident           3357.0104  279.6443  12.0046 0.0000  2808.8998  3905.1209
ext_col_processed  -707.0567  163.7445  -4.3180 0.0000 -1028.0005  -386.1129
horsepower          134.5561    2.8875  46.5999 0.0000   128.8965   140.2156
displacement        -29.5626   15.8612  -1.8638 0.0624   -60.6510     1.5259
cylinder          -1081.7411  127.3620  -8.4934 0.0000 -1331.3740  -832.1082
type                237.8249   38.3643   6.1991 0.0000   162.6299   313.0200
fuel                102.9733   92.8166   1.1094 0.2673   -78.9498   284.8963
----------------------------------------------------------------------------
Omnibus:                29478.213        Durbin-Watson:           2.003     
Prob(Omnibus):          0.000            Jarque-Bera (JB):        798249.047
Skew:                   3.637            Prob(JB):                0.000     
Kurtosis:               24.426           Condition No.:           1966      
============================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is
correctly specified.
[2] The condition number is large, 1.97e+03. This might indicate
that there are strong multicollinearity or other numerical
problems.
"""

In [170]:
train_processed = train_processed.drop(['fuel','displacement'],axis=1)
test_processed = test_processed.drop(['fuel','displacement'],axis=1)

In [171]:
# train_processed = train_processed.drop(['fuel'],axis=1)
# test_processed = test_processed.drop(['fuel'],axis=1)

In [172]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb

def evaluate_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the models
    models = {
        'Linear Regression': LinearRegression(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor(),
        'Gradient Boosting': GradientBoostingRegressor(),
        'AdaBoost': AdaBoostRegressor(),
        'K-Nearest Neighbors': KNeighborsRegressor(),
        'Neural Network': MLPRegressor(max_iter=1000),
        'XGBoost': xgb.XGBRegressor(),
        'LightGBM': lgb.LGBMRegressor()
    }

     # Train and evaluate each model
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        print(f'{name}: RMSE = {rmse:.4f}, R² = {r2:.4f}')

X = train_processed.drop(columns='price') 
y = train_processed['price']
evaluate_models(X, y)

Linear Regression: RMSE = 22683.7547, R² = 0.3976
Decision Tree: RMSE = 29428.1832, R² = -0.0139
Random Forest: RMSE = 22627.0488, R² = 0.4006
Gradient Boosting: RMSE = 21205.0569, R² = 0.4736
AdaBoost: RMSE = 24833.5227, R² = 0.2780
K-Nearest Neighbors: RMSE = 23089.1053, R² = 0.3759
Neural Network: RMSE = 21903.2854, R² = 0.4383
XGBoost: RMSE = 21730.6004, R² = 0.4471
LightGBM: RMSE = 21148.1209, R² = 0.4764


In [173]:
model = lgb.LGBMRegressor()
X = train_processed.drop(columns='price') 
y = train_processed['price']
model.fit(X, y)

LGBMRegressor()

In [174]:
test_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model_year         36183 non-null  float64
 1   milage             36183 non-null  float64
 2   int_black          36183 non-null  float64
 3   brand              36183 non-null  int32  
 4   accident           36183 non-null  int32  
 5   ext_col_processed  36183 non-null  int32  
 6   horsepower         36183 non-null  int32  
 7   cylinder           36183 non-null  int32  
 8   type               36183 non-null  int32  
 9   id                 36183 non-null  int64  
dtypes: float64(3), int32(6), int64(1)
memory usage: 1.9 MB


In [175]:
test = test_processed.drop(['id'],axis=1)
y_pred = model.predict(test)

In [176]:
y_pred

array([27959.22403884, 21418.48656043, 33200.83999745, ...,
        9958.65079321, 52402.10726297, 12925.58940639])

In [177]:
test_processed['id']

0        54273
1        54274
2        54275
3        54276
4        54277
         ...  
36178    90451
36179    90452
36180    90453
36181    90454
36182    90455
Name: id, Length: 36183, dtype: int64

In [178]:
result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)

In [179]:
from sklearn.model_selection import GridSearchCV
lgbm = lgb.LGBMRegressor()

# Define the parameter grid
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

X = train_processed.drop(columns='price') 
y = train_processed['price']

# Fit the model
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(test)

Best parameters found: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200, 'num_leaves': 31}


In [180]:
result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('better_predictions.csv', index=False)

In [181]:
# from sklearn.ensemble import StackingRegressor

# stacking_reg = StackingRegressor(
#     estimators = [
#         ('Extra Trees ',xgb.XGBRegressor()),
#         ('Random Forest ',lgb.LGBMRegressor()),
#         ('Linear Regression', GradientBoostingRegressor())
#     ],
#     final_estimator = lgb.LGBMRegressor() , cv = 5
# )
# stacking_reg.fit(X , y)
# y_pred = stacking_reg.predict(test)

# # rmse = np.sqrt(mean_squared_error(y_test , y_pred))
# # print(f"Root mean squared error of the model : {rmse} ")
# # r2 = r2_score(y_test , y_pred)
# # print(f"R2 score of the model : {r2} ")

In [182]:
# result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# # Save the result DataFrame to a CSV file
# result_df.to_csv('stack_predictions.csv', index=False)

In [183]:
model = GradientBoostingRegressor()
X = train_processed.drop(columns='price') 
y = train_processed['price']
model.fit(X, y)

y_pred = best_model.predict(test)
result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('gdb_predictions.csv', index=False)